### Importing packages and setting FILE_NAMES

In [15]:
import numpy as np
import pandas as pd

years = [2019,2021,2022,2023,2024]

FILE_NAMES = [
    f'../../data/raw/HMS_{year}-{year+1}_PUBLIC_instchars.csv' for year in years
]

In [ ]:
race_cols = ['race_black', 'race_ainaan', 'race_asian', 'race_his', 'race_pi', 'race_mides', 'race_white']
# data in audit_9 is corrupted
alcohol_in_substance_module = [f'audit_{i}' for i in range(1,11) if i != 9]

categories_with_int_values = ['international','alc_any','alc_binge'] + alcohol_in_substance_module + ['smok_freq','smok_vape'] + ['exerc']

for i,year in enumerate(years):
    new_filename = f'../../data/interim/{year}-{year+1}_jose.csv'
    df = pd.read_csv(FILE_NAMES[i])

    # All years except 2021-2022 have an aggregate binge_fr feature.
    # For 2021-2022 need to aggregate q3_22 (which should be binge_fr_f), binge_fr_m, and q3_22_0 (which should be binge_fr_o)

    if year == 2021:
        df['alc_binge'] = df[['q3_22','binge_fr_m','q3_22_0']].astype('Int64').max(axis=1,skipna=True)
    else: 
        df['alc_binge'] = df['binge_fr']

    # smok_freq and smok_vape are missing in the last year

    if year == 2024:
        df['smok_freq'] = np.nan
        df['smok_vape'] = np.nan
    else:
        # Encoding changed from 1=yes 2=no to 1=yes 0=no
        df['smok_vape'] = df['smok_vape'] % 2

    # exerc

    if year == 2022:
        df['exerc'] = df['exerc_range4']
    elif year == 2021:
        df['exerc'] = df['exerc'].replace(to_replace=6, value=2)
    elif year == 2023 or year == 2024:
        df['exerc'] = df['exerc'].replace(to_replace=[3,4,5], value=[2,3,4])

    # Encoding race in one columns
    # Flag entries marking no race or multiple races

    df['race'] = df[race_cols].fillna(0).dot([1,2,3,4,5,6,7]).astype('Int64')
    df['will_remove'] = (df[race_cols].fillna(0).sum(axis =1) != 1).astype('Int64')

    for feature in categories_with_int_values:
        df[feature] = df[feature].astype('Int64')

    df = df[['responseid' , 'will_remove', 'race'] + categories_with_int_values]

    df.to_csv(new_filename , index=False)

/tmp/ipykernel_15862/3963266800.py:9: DtypeWarning: Columns (8,16,18,29,31,81,85,113,151,166,218,255,278,304,309,317,324,329,335,343,351,364,368,385,399,400,456,462,469,479,497,498,499,528,561,607,657,667,668,686,720,728,745,749,770,797,813,814,818,857,943,956,965,975,1024,1055,1056,1090,1104,1117,1154,1197,1199,1292,1295,1309,1318,1328,1329,1330,1352,1364,1365,1366,1367,1374,1385,1398,1404,1413,1416,1431,1451,1466,1467,1468,1469,1470,1474,1477,1480,1482,1483,1487,1488,1489,1491,1495,1497,1499,1502,1519,1541,1544,1568,1592,1619,1685,2332,2334,2339) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(FILE_NAMES[i])
/tmp/ipykernel_15862/3963266800.py:9: DtypeWarning: Columns (13,24,39,45,52,74,96,98,102,151,171,188,219,222,345,353,360,365,371,379,387,400,405,417,432,515,521,527,533,539,545,551,558,567,588,589,612,624,657,674,675,728,758,759,766,785,791,800,814,848,865,897,902,904,923,1005,1038,1039,1040,1041,1059,1089,1105,1118,1126,1176,1215,1232